In [3]:
import pandas as pd
import spacy 
import nltk
import math 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import re

In [69]:
txt  =  """
 Google and Facebook are strangling the free press to death. Democracy is the loserGoogle an 
Your 60-second guide to security stuff Google touted today at Next '18
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo’s Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is
Android is better than IOS
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
is a numerical statistic that is intended to reflect
how important a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval
text mining, and user modeling. The tf-idf value increases proportionally
to the number of times a word appears in the document
and is offset by the frequency of the word in the corpus
"""

In [38]:
corpus = """
Simple example with Cats and Mouse
Another simple example with dogs and cats
Another simple example with mouse and cheese
"""

In [39]:
corpus = corpus.split('\n')[1:-1]
la = corpus[0].lower().strip().split(' ')
lb = corpus[1].lower().strip().split(' ')
lc = corpus[2].lower().strip().split(' ')
uq = set(la).union(set(lb)).union(set(lc))
w1 = dict.fromkeys(uq,0)
w2 = dict.fromkeys(uq,0)
w3 = dict.fromkeys(uq,0)

In [40]:
la

['simple', 'example', 'with', 'cats', 'and', 'mouse']

In [41]:
def counterizer(l,w):
    for x in l:
        w[x] += 1
    return w

In [42]:
w1_dict = counterizer(la,w1)
w2_dict = counterizer(lb,w2)
w3_dict = counterizer(lc,w3)

In [43]:
w1_dict

{'dogs': 0,
 'example': 1,
 'and': 1,
 'another': 0,
 'cats': 1,
 'cheese': 0,
 'with': 1,
 'simple': 1,
 'mouse': 1}

In [48]:
def tf(vec,l):
    n = float(len(l))
    tf = {}
    for k,v in vec.items():
        tf[k] = v / n
    return tf

In [51]:
tf_a = tf(w1_dict,la)

In [52]:
tf_b = tf(w2_dict,lb)

In [53]:
tf_c = tf(w3_dict,lc)

In [54]:
tf_a

{'dogs': 0.0,
 'example': 0.16666666666666666,
 'and': 0.16666666666666666,
 'another': 0.0,
 'cats': 0.16666666666666666,
 'cheese': 0.0,
 'with': 0.16666666666666666,
 'simple': 0.16666666666666666,
 'mouse': 0.16666666666666666}

In [55]:
tf_b

{'dogs': 0.14285714285714285,
 'example': 0.14285714285714285,
 'and': 0.14285714285714285,
 'another': 0.14285714285714285,
 'cats': 0.14285714285714285,
 'cheese': 0.0,
 'with': 0.14285714285714285,
 'simple': 0.14285714285714285,
 'mouse': 0.0}

In [56]:
tf_c

{'dogs': 0.0,
 'example': 0.14285714285714285,
 'and': 0.14285714285714285,
 'another': 0.14285714285714285,
 'cats': 0.0,
 'cheese': 0.14285714285714285,
 'with': 0.14285714285714285,
 'simple': 0.14285714285714285,
 'mouse': 0.14285714285714285}

In [61]:
def idf(docs):
    idf = dict.fromkeys(docs[0].keys(),0)
    n = float(len(docs))
    for doc in docs:
        for k,v in doc.items():
            if v > 0:
                idf[k] += 1

    for k,v in idf.items():
        idf[k] = math.log(n/idf[k])
    return idf

In [62]:
idf = idf([w1_dict,w2_dict,w3_dict])
idf

{'dogs': 1.0986122886681098,
 'example': 0.0,
 'and': 0.0,
 'another': 0.4054651081081644,
 'cats': 0.4054651081081644,
 'cheese': 1.0986122886681098,
 'with': 0.0,
 'simple': 0.0,
 'mouse': 0.4054651081081644}

In [63]:
def tf_idf(tf,idf):
    tf_idf = dict.fromkeys(tf.keys(),0)
    for k,v in tf.items():
        tf_idf[k] = v * idf[k]
    return tf_idf

In [64]:
tf_idf_a = tf_idf(tf_a,idf)
tf_idf_a

{'dogs': 0.0,
 'example': 0.0,
 'and': 0.0,
 'another': 0.0,
 'cats': 0.06757751801802739,
 'cheese': 0.0,
 'with': 0.0,
 'simple': 0.0,
 'mouse': 0.06757751801802739}

In [65]:
tf_idf_b = tf_idf(tf_b,idf)
tf_idf_b

{'dogs': 0.15694461266687282,
 'example': 0.0,
 'and': 0.0,
 'another': 0.05792358687259491,
 'cats': 0.05792358687259491,
 'cheese': 0.0,
 'with': 0.0,
 'simple': 0.0,
 'mouse': 0.0}

In [66]:
tf_idf_c = tf_idf(tf_c,idf)
tf_idf_c

{'dogs': 0.0,
 'example': 0.0,
 'and': 0.0,
 'another': 0.05792358687259491,
 'cats': 0.0,
 'cheese': 0.15694461266687282,
 'with': 0.0,
 'simple': 0.0,
 'mouse': 0.05792358687259491}

In [71]:
txt = txt.split('\n')[1:-1]
txt

[' Google and Facebook are strangling the free press to death. Democracy is the loserGoogle an ',
 "Your 60-second guide to security stuff Google touted today at Next '18",
 'A Guide to Using Android Without Selling Your Soul to Google',
 'Review: Lenovo’s Google Smart Display is pretty and intelligent',
 'Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is',
 'Android is better than IOS',
 'In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency',
 'is a numerical statistic that is intended to reflect',
 'how important a word is to a document in a collection or corpus.',
 'It is often used as a weighting factor in searches of information retrieval',
 'text mining, and user modeling. The tf-idf value increases proportionally',
 'to the number of times a word appears in the document',
 'and is offset by the frequency of the word in the corpus']

In [72]:
def preprocess(txt):
    for i in range(len(txt)):
        txt[i] = txt[i].lower().strip()
    return txt

In [73]:
preprocess(txt)

['google and facebook are strangling the free press to death. democracy is the losergoogle an',
 "your 60-second guide to security stuff google touted today at next '18",
 'a guide to using android without selling your soul to google',
 'review: lenovo’s google smart display is pretty and intelligent',
 'google maps user spots mysterious object submerged off the coast of greece - and no-one knows what it is',
 'android is better than ios',
 'in information retrieval, tf–idf or tfidf, short for term frequency–inverse document frequency',
 'is a numerical statistic that is intended to reflect',
 'how important a word is to a document in a collection or corpus.',
 'it is often used as a weighting factor in searches of information retrieval',
 'text mining, and user modeling. the tf-idf value increases proportionally',
 'to the number of times a word appears in the document',
 'and is offset by the frequency of the word in the corpus']

In [77]:
# nlp = spacy.load("en_core_web_sm")
# nlp.Defaults.stop_words.remove("not")
# nlp.vocab["not"].is_stop = False
def process(txt):
    pro_txt = []
    for i in range(len(txt)):
        doc = nlp(txt[i])
        l = []
        for tok in doc:
            if not (tok.is_punct or tok.is_stop or tok.is_space):
                l.append(tok.lemma_)
        pro_txt.append(" ".join(l))
    return pro_txt

In [79]:
txt1 = process(txt)

In [80]:
tf_idf = TfidfVectorizer()
tf_txt = tf_idf.fit_transform(txt1)

In [81]:
kmeans = KMeans(n_clusters=2).fit(tf_txt)

C:\Users\ELmohannd\anaconda3\envs\tensorflow3\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\ELmohannd\anaconda3\envs\tensorflow3\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [82]:
lines_for_predicting = ["tf and idf is awesome!", "Selling Your Soul to Google"]
kmeans.predict(tf_idf.transform(lines_for_predicting))

array([0, 1])